# МФТИ: МТИИ 2020 Python.

## Семинар 10: Cython. PyTorch. 

## Cython

Cython $-$ это Python, с поддержкой типов данных языка C.
Практически любой фрагмент кода Python также является допустимым кодом Cython. (Есть несколько ограничений.) Компилятор Cython преобразует python в код C, который выполняет те же вызовы API Python/C.

Благодаря Cython мы можем смешивать код Python и C, 
Поддержка счетчика ссылок и проверка ошибок выполняется автоматически, и нам доступны все возможности Python для обработки исключений, включая операторы try-except - даже в процессе манипулирования данными C.

### Cython Hello World


Поскольку Cython может принимать практически любой  исходный код на Python, одна из самых сложных задач при запуске $-$ это просто выяснить, как этот код скомпилировать.

Давайте начнем с классического ```Hello World```. Создадим файл ```helloworld.pyx``` со следующим кодом внутри:
```python
print("Hello World!")
```

Сделать это на linux или colab можно с помощью следующей команды:

In [1]:
!mkdir hello_world -p
!echo "print(\"Hello World!\")" > hello_world/helloworld.pyx

In [3]:
!cd hello_world && ls

helloworld.pyx	setup.py


Теперь создадим файл ```setup.py```, который нужен нам для компиляции. Запишем в него следующий код:

```python
from setuptools import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("helloworld.pyx")
)
```

Теперь, чтобы собрать Cython файл нужно написать команду:

In [4]:
!cd hello_world && python3 setup.py build_ext --inplace

Compiling helloworld.pyx because it changed.
[1/1] Cythonizing helloworld.pyx
/home/tv-home/.local/lib/python3.8/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/tv-home/Desktop/mipt/2020/python/seminar-10/hello_world/helloworld.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'helloworld' extension
creating build
creating build/temp.linux-x86_64-3.8
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c helloworld.c -o build/temp.linux-x86_64-3.8/helloworld.o
creating build/lib.linux-x86_64-3.8
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl

В результате в текущей директории появился файл helloworld.so (helloworld.pyd в Windows). Теперь, для того, чтобы его запустить нам нужно импортировать его, как обычный python модуль. 

In [5]:
from hello_world import helloworld

Hello World!


Отлично! Теперь мы знаем, как создать и скомпилировать код, с помощью Cython.  

### Fibonacci
Рассмотрим пример интереснее. В официальном руководстве Python есть вот такая функция для печати чисел Фибоначчи:
```python
from __future__ import print_function

def fib(n):
    """Print the Fibonacci series up to n."""
    a, b = 0, 1
    while b < n:
        print(b, end=' ')
        a, b = b, a + b

    print()
```


Теперь, следуя шагам для примера c "Hello World", мы сначала переименовываем файл, чтобы он имел расширение ```.pyx```, скажем, ```fib.pyx```, а затем создаем файл ```setup.py```. 

In [6]:
!cd fib && python3 setup.py build_ext --inplace

running build_ext
copying build/lib.linux-x86_64-3.8/fib.cpython-38-x86_64-linux-gnu.so -> 


In [7]:
from fib.fib import fib

fib(2020)

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 


Хорошо, мы разобрали пример интереснее, единственная проблема $-$ мы не поняли, зачем же нам вообще нужно что-то компилировать.

### Primes


Вот небольшой пример, показывающий, зачем нам нужен Cython. Есть обычная функция, для поиска простых чисел. Вы сообщаете ей, сколько простых чисел вы хотите, и она возвращает их в виде списка:

```cython
def primes(int nb_primes):
    cdef int n, i, len_p
    cdef int p[1000]
    if nb_primes > 1000:
        nb_primes = 1000

    len_p = 0  # The current number of elements in p.
    n = 2
    while len_p < nb_primes:
        # Is n prime?
        for i in p[:len_p]:
            if n % i == 0:
                break

        # If no break occurred in the loop,
        # we have a prime.
        else:
            p[len_p] = n
            len_p += 1
        n += 1

    # Let's return the result in a python list:
    result_as_list  = [prime for prime in p[:len_p]]
    return result_as_list
```


Можно заметить, что код начинается так же, как при определении обычной функции, за исключением того, что параметр nb_primes объявлен с типом int. Это означает, что переданный объект будет преобразован в целое число C (или будет получена ошибка TypeError, если это невозможно).

Обратим внимание на строки:
```python
cdef int n, i, len_p
cdef int p[1000]
```

Обратим внимание на массив ```p```, он будет преобразован в список после выполнения 22-ой строки ``` return result_as_list```.

*Примечание. Очень большие массивы таким образом создать не получится, т.к. они размещаются в стеке функций C (function call stack), который является довольно ценным и дефицитным ресурсом. Как запросить массивы большей размерности в Cython  вы можете узнать здесь: [C memory allocation](https://cython.readthedocs.io/en/latest/src/tutorial/memory_allocation.html#memory-allocation), [memory_allocation](https://cython.readthedocs.io/en/latest/src/tutorial/array.html#array-array) и [NumPy arrays](https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html#memoryviews).*

In [8]:
!cd primes && python3 setup.py build_ext --inplace

running build_ext
copying build/lib.linux-x86_64-3.8/primes.cpython-38-x86_64-linux-gnu.so -> 


In [11]:
from primes.primes import primes

primes(10)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

Сравним эту функцию со следующей:

In [12]:
def primes_slow(nb_primes):
    p = []
    n = 2
    while len(p) < nb_primes:
        # Is n prime?
        for i in p:
            if n % i == 0:
                break

        # If no break occurred in the loop
        else:
            p.append(n)
        n += 1
    return p

In [15]:
%time primes(10**5)
pass

CPU times: user 3.33 ms, sys: 0 ns, total: 3.33 ms
Wall time: 4.09 ms


In [16]:
%time primes_slow(10**5)
pass

CPU times: user 2min 43s, sys: 43 ms, total: 2min 43s
Wall time: 2min 43s


Также, мы могли бы добавить к сравнению ```primes_slow``` скомпилированную с помощью Cython.

### Primes & C++

Cython дает нам возможность использовать преимущества языка C++, в частности, часть стандартной библиотеки C++ напрямую импортируется из кода Cython.

Давайте посмотрим, во что превратится ```primes.pyx``` при использовании vector, из стандартной библиотеки C++.

Создадим файл ```cpp/cpp_primes.pyx``` со следующим кодом: 

```cython
# distutils: language=c++

from libcpp.vector cimport vector

def primes(unsigned int nb_primes):
    cdef int n, i
    cdef vector[int] p
    # allocate memory for 'nb_primes' elements.
    p.reserve(nb_primes)  

    n = 2
    # size() for vectors is similar to len()
    while p.size() < nb_primes:  
        for i in p:
            if n % i == 0:
                break
        else:
            # push_back is similar to append()
            p.push_back(n)  
        n += 1

    # Vectors are automatically converted to Python
    # lists when converted to Python objects.
    return p
```

In [17]:
!cd cpp && python3 setup.py build_ext --inplace

running build_ext
copying build/lib.linux-x86_64-3.8/cpp_primes.cpython-38-x86_64-linux-gnu.so -> 


In [19]:
from cpp.cpp_primes import primes as cpp_primes

cpp_primes(10)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

In [20]:
%time cpp_primes(10**5)
pass

CPU times: user 17.6 s, sys: 28 ms, total: 17.7 s
Wall time: 17.6 s


Дополнительная литература:

* [Cython Tutorials](https://cython.readthedocs.io/en/latest/src/tutorial/index.html)